In [1]:
from network_evaluation_tools import gene_conversion_tools as gct
import pandas as pd
import itertools

## Load PID Raw Data
#### Source (MITAB): http://dip.doe-mbi.ucla.edu/dip/File.cgi?FN=2016/tab25/Hsapi20170205.txt
Downloaded: June 15, 2017  
Last Updated: Februrary 05, 2017    
Notes for download: Website requires registration. Register for the site to download the file from the link.  
Notes for processing: This is the file for human protein interactions, however, not all interactions are human-human interactions. These need to be filtered. Also all ID's not without RefSeq or UniProt ID are excluded. Custom processing for this network is described below

In [5]:
wd = '/cellar/users/jkhuang/Data/Projects/Network_Analysis/Data/'
DIP_Raw = pd.read_csv(wd+'Network_Data_Raw/DIP/Hsapi20170205.txt', index_col=0, sep='\t')
print 'Raw edge count in DIP:', len(DIP_Raw)

Raw edge count in DIP: 7794


In [12]:
# Fix the column offset in the interaction data table
DIP_Raw_offset = DIP_Raw.reset_index(drop=False)[DIP_Raw.reset_index(drop=False).columns[:-2]]
DIP_Raw_offset.columns = DIP_Raw.columns[:-1]

In [16]:
# Keep only human-human interactions
DIP_Human_only = DIP_Raw_offset[(DIP_Raw_offset['Taxid interactor A']=='taxid:9606(Homo sapiens)') & (DIP_Raw_offset['Taxid interactor B']=='taxid:9606(Homo sapiens)')]
print 'Human-Human only interactions in DIP:', len(DIP_Human_only)

Human-Human only interactions in DIP: 5569


#### Parse all genes in filtered DIP and keep only RefSeq/UniProtKB labelled interactions

In [18]:
# Extract gene list
Human_DIP_Genes = list(set(DIP_Human_only['ID interactor A']).union(set(DIP_Human_only['ID interactor B'])))

In [25]:
# Split all gene names into list of genes and concatenate
Human_DIP_Genes_split = [name.split('|') for name in Human_DIP_Genes]
Human_DIP_Genes_full_list = list(itertools.chain.from_iterable(Human_DIP_Genes_split))

# Note about this line: This is to fix the one example where one of the Uniprot genes gets labelled as "uniprotkb:Q13936,159'
Human_DIP_Genes_full_list = [name.split(',')[0] for name in Human_DIP_Genes_full_list] 

## Convert Genes

In [26]:
# Construct list of genes to be submitted to MyGene.Info API (remove all genes with 'DIP' prefix)
query_string, valid_genes, invalid_genes = gct.query_constructor(Human_DIP_Genes_full_list, exclude_prefixes=['DIP'])

5017 Valid Query Genes
3281 Invalid Query Genes


In [31]:
# Set scopes (gene naming systems to search)
scopes = "uniprot, refseq"
# Set fields (systems from which to return gene names from)
fields = "symbol, entrezgene"
# Query MyGene.Info
match_list = gct.query_batch(query_string, scopes=scopes, fields=fields)
print len(match_list), 'Matched query results'

Batch query complete: 7.97 seconds
5074 Matched query results


In [32]:
match_table_trim, query_to_symbol, query_to_entrez = gct.construct_query_map_table(match_list, valid_genes)

Queries without full matching results found: 106

74 Queries with mutliple matches found

Query mapping table/dictionary construction complete: 6.82 seconds


## Construct Converted Network

In [36]:
# This is a custom gene conversion function written due to the parsing required for gene interactor labels
# Returns best matched symbol and/or entrez id from each DIP interactor string (if applicable)
def convert_DIP_string(string, field):
    names = [gct.get_identifier_without_prefix(name) for name in string.split('|')]
    # Keep only mappings defined for field of interest
    if field=='symbol':
        # Return match table values that have matched symbol
        conversion = match_table_trim.ix[names][~(match_table_trim.ix[names]['Symbol'].isnull())]
        # Return conversion with max score or None if no conversion
        if conversion.shape[0]==0:
            return None
        else:
            max_score = conversion['Score'].max()
            return conversion[conversion['Score']==max_score].ix[0]['Symbol']
    elif field=='entrez':
        # Return match table values that have matched symbol
        conversion = match_table_trim.ix[names][~(match_table_trim.ix[names]['EntrezID'].isnull())]
        if conversion.shape[0]==0:
            return None
        else:
            # Return conversion with max score or None if no conversion
            max_score = conversion['Score'].max()
            return conversion[conversion['Score']==max_score].ix[0]['EntrezID']

In [37]:
DIP_Human_only_edges = DIP_Human_only[['ID interactor A', 'ID interactor B']].values.tolist()
DIP_edgelist_symbol = [sorted([convert_DIP_string(edge[0],'symbol'),convert_DIP_string(edge[1],'symbol')]) for edge in DIP_Human_only_edges]

In [39]:
# Filter converted edge list
DIP_edgelist_symbol_filt = gct.filter_converted_edgelist(DIP_edgelist_symbol)

5569 input edges
512 self-edges removed
309 edges with un-mapped genes removed
26 duplicate edges removed
Edge list filtered: 0.02 seconds
4722 Edges remaining


In [40]:
# Save converted edge list
gct.write_edgelist(DIP_edgelist_symbol_filt, wd+'Network_SIFs_Symbol/DIP_Symbol.sif')

Edge list saved: 0.02 seconds
